In [1]:
!pip install scikit-fuzzy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 14.7 MB/s eta 0:00:00


In [3]:
import random
import smtplib
from email.mime.text import MIMEText
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import logging
import os

# إعداد تسجيل الدخول
logging.basicConfig(filename='fire_detection.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 1. جمع البيانات من المستشعرات (باستخدام قيم عشوائية منطقية لمنطقة عسير)
def get_sensor_data():
    temperature = random.uniform(20.0, 40.0)  # درجة الحرارة بين 20 و 40 درجة مئوية
    humidity = random.uniform(40.0, 80.0)     # الرطوبة بين 40% و 80%
    smoke_level = random.uniform(0.0, 10.0)   # مستوى الدخان بين 0 و 10
    wind_speed = random.uniform(5.0, 15.0)    # سرعة الرياح بين 5 و 15 كم/ساعة
    drought = random.uniform(0.3, 0.8)        # الجفاف بين 0.3 و 0.8
    location_risk = random.uniform(0.4, 0.7)  # مستوى المخاطر في الموقع بين 0.4 و 0.7
    return temperature, humidity, smoke_level, wind_speed, drought, location_risk

# 2. إعداد المتغيرات الضبابية
temperature = ctrl.Antecedent(np.arange(20, 41, 1), 'temperature')
humidity = ctrl.Antecedent(np.arange(40, 81, 1), 'humidity')
smoke_level = ctrl.Antecedent(np.arange(0, 11, 1), 'smoke_level')
wind_speed = ctrl.Antecedent(np.arange(5, 16, 1), 'wind_speed')
drought = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'drought')
location_risk = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'location_risk')
fire_risk = ctrl.Consequent(np.arange(0, 11, 1), 'fire_risk')

# تعريف دوال الانتماء الضبابية
temperature.automf(3)
humidity.automf(3)
smoke_level.automf(3)
wind_speed.automf(3)
drought.automf(3)
location_risk.automf(3)

fire_risk['low'] = fuzz.trimf(fire_risk.universe, [0, 0, 5])
fire_risk['medium'] = fuzz.trimf(fire_risk.universe, [0, 5, 10])
fire_risk['high'] = fuzz.trimf(fire_risk.universe, [5, 10, 10])

# قواعد منطق الضبابية
rule1 = ctrl.Rule(temperature['poor'] & humidity['poor'] & smoke_level['good'] & wind_speed['good'] & drought['poor'] & location_risk['good'], fire_risk['high'])
rule2 = ctrl.Rule(temperature['average'] & humidity['average'] & smoke_level['average'] & wind_speed['average'] & drought['average'] & location_risk['average'], fire_risk['medium'])
rule3 = ctrl.Rule(temperature['good'] & humidity['good'] & smoke_level['poor'] & wind_speed['poor'] & drought['good'] & location_risk['poor'], fire_risk['low'])

# إنشاء نظام التحكم الضبابي
fire_risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
fire_risk_sim = ctrl.ControlSystemSimulation(fire_risk_ctrl)

# 3. تحليل البيانات باستخدام منطق الضبابية
def analyze_data_with_fuzzy(temperature_val, humidity_val, smoke_level_val, wind_speed_val, drought_val, location_risk_val):
    fire_risk_sim.input['temperature'] = temperature_val
    fire_risk_sim.input['humidity'] = humidity_val
    fire_risk_sim.input['smoke_level'] = smoke_level_val
    fire_risk_sim.input['wind_speed'] = wind_speed_val
    fire_risk_sim.input['drought'] = drought_val
    fire_risk_sim.input['location_risk'] = location_risk_val
    fire_risk_sim.compute()

    # فحص مخرجات منطق الضبابية
    if 'fire_risk' in fire_risk_sim.output:
        return fire_risk_sim.output['fire_risk']
    else:
        print("Error: 'fire_risk' key not found in output. No rules were triggered.")
        return None

# 4. إرسال الإشعارات في حال اكتشاف حريق محتمل
def send_alert(location):
    msg = MIMEText(f"Potential fire detected at {location}. Please respond immediately!")
    msg['Subject'] = 'Forest Fire Alert'
    msg['From'] = os.getenv('EMAIL_FROM')
    msg['To'] = os.getenv('EMAIL_TO')

    try:
        with smtplib.SMTP(os.getenv('SMTP_SERVER'), int(os.getenv('SMTP_PORT'))) as server:
            server.starttls()
            server.login(os.getenv('EMAIL_FROM'), os.getenv('EMAIL_PASSWORD'))
            server.send_message(msg)
            logging.info("Alert sent successfully!")
    except Exception as e:
        logging.error(f"Failed to send alert: {e}")

# 5. تجميع المشروع
def main():
    location = "Asir Region,Location XYZ"

    # جمع البيانات
    temperature, humidity, smoke_level, wind_speed, drought, location_risk = get_sensor_data()
    logging.info(f"Temperature: {temperature:.2f}°C, Humidity: {humidity:.2f}%, Smoke Level: {smoke_level:.2f}, Wind Speed: {wind_speed:.2f} km/h, Drought: {drought:.2f}, Location Risk: {location_risk:.2f}")

    # تحليل البيانات باستخدام منطق الضبابية
    fire_risk_value = analyze_data_with_fuzzy(temperature, humidity, smoke_level, wind_speed, drought, location_risk)
    print(f"Temperature: {temperature:.2f}°C")
    print(f"Humidity: {humidity:.2f}%")
    print(f"Smoke Level: {smoke_level:.2f}")
    print(f"Wind Speed: {wind_speed:.2f} km/h")
    print(f"Drought: {drought:.2f}")
    print(f"Location Risk: {location_risk:.2f}")

    if fire_risk_value is not None:
        print(f"Fire Risk Value: {fire_risk_value:.2f}")
        logging.info(f"Fire risk value: {fire_risk_value:.2f}")

        # تحديد حالة الخطر بناءً على عتبة
        if fire_risk_value > 7.0:
            logging.warning("Potential fire detected!")
            print("Warning: Potential fire detected!")
            send_alert(location)
        else:
            logging.info("No fire detected.")
            print("No fire detected.")
    else:
        print("Failed to compute fire risk value.")

if __name__ == "__main__":
    main()

Temperature: 22.09°C
Humidity: 74.22%
Smoke Level: 5.65
Wind Speed: 7.35 km/h
Drought: 0.76
Location Risk: 0.52
Fire Risk Value: 5.00
No fire detected.
